In [2]:
import pandas as pd
import nltk
import torch
from torchtext import data
from torchtext.data import TabularDataset
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [3]:
train = pd.read_csv('./train15k.csv')
validation = pd.read_csv('./validation4k.csv')
test = pd.read_csv('./test579.csv')

In [4]:
t = pd.read_csv('./train15k.csv')
t.head(3)

,text,author
0,"This process, however, afforded me no means of...",EAP
1,It never once occurred to me that the fumbling...,HPL
2,"In his left hand was a gold snuff box, from wh...",EAP


In [5]:
v = pd.read_csv('./validation4k.csv')
v.head(3)

,text,author
0,It lay beneath me like a chart and although I ...,EAP
1,"I asked, rubbing my hands over the blaze, and ...",EAP
2,"Fire, the sword, and plague They may all be fo...",MWS


In [6]:
def gen_bigram(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [7]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = ('cuda' if torch.cuda.is_available() else 'cpu')

Text = data.Field(tokenize = 'spacy')
Label = data.LabelField()

In [8]:
%%time

train, validation = TabularDataset.splits(
path = './', format='csv', skip_header = True, train = './train15k.csv', validation = './validation4k.csv',
fields = [('text', Text), ('author', Label)]
)

CPU times: user 19.1 s, sys: 251 ms, total: 19.4 s
Wall time: 20.2 s


In [9]:
%%time
max_vocab_size = 30_000

Text.build_vocab(train,
                max_size = max_vocab_size,
                vectors = 'glove.6B.100d',
                unk_init = torch.Tensor.normal_
                )
Label.build_vocab(train)

CPU times: user 719 ms, sys: 230 ms, total: 949 ms
Wall time: 1.03 s


In [10]:
train_iter, val_iter = data.BucketIterator.splits(
    (train, validation),
    sort_key = lambda x: len(x.text),
    batch_size = 64,
    device = device
    )

In [11]:
class FastText(nn.Module):
    def __init__(self, vocab_size, emb_dim, padding_index, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx = padding_index)
        self.fc = nn.Linear(emb_dim, output_dim)
        
    def forward(self, text):
        # text = [sequence len X BS]
        embedded = self.embedding(text)
        
        # embedded = [sequence len X BS X emb dim]
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [BS X seq len X emb dim]
        pooled_layer = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)
        
        # pooled_layer = [BS X emb dim]
        return self.fc(pooled_layer)

In [14]:
vocab_size = len(Text.vocab)
emb_dim = 100
output_dim = len(Label.vocab)
padding_index = Text.vocab.stoi[Text.pad_token]

model = FastText(vocab_size, emb_dim, padding_index, output_dim)

In [15]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
count_params(model)

2575403

In [17]:
pretrained_embeddings = Text.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
        ...,
        [ 0.3800, -0.4023, -0.1590,  ..., -0.2469, -0.2327,  0.0116],
        [ 1.1719, -1.7597, -0.2668,  ..., -2.4071, -0.0571,  1.5848],
        [ 0.2093,  1.6408, -0.4784,  ..., -0.0414,  0.4682, -0.8824]])

In [18]:
unk_idx = Text.vocab.stoi[Text.unk_token]
unk_idx

0

In [22]:
model.embedding.weight.data[unk_idx] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [23]:
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:
optimizer = optim.Adam(model.parameters())

In [26]:
def accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim=True)
    correct = max_preds.squeeze(1).eq(y)
#     print('this is correct', correct.sum().item() / 64)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [27]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
#         print('1')        
        optimizer.zero_grad()
#         print('2')
        predictions = model(batch.text).squeeze(1)
#         print('predictions', predictions.argmax(1, keepdim=True))
#         print('real', batch.author)
#         print('3')        
        loss = criterion(predictions, batch.author)
#         print('4')
        acc = accuracy(predictions, batch.author)
#         print('5')
        loss.backward()

#         print('6')
        optimizer.step()
#         print('7')
        
        epoch_loss += loss.item()
        epoch_acc = acc.item()
#         print(epoch_acc)

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.author)
            
            acc = categorical_accuracy(predictions, batch.author)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
%%time
n_epochs = 5

for epoch in range(n_epochs):
    
#     train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = train(model, train_iter, optimizer, criterion)
        
#     print(f'Train loss: {train_loss:.3f}, train_acc: {train_acc * 100:.3f}')
    print(f'valid loss: {valid_loss:.3f}, train_acc: {valid_acc * 100:.3f}')    